# Aviation Accidents Analysis

You are part of a consulting firm that is tasked to do an analysis of commercial and passenger jet airline safety. The client (an airline/airplane insurer) is interested in knowing what types of aircraft (makes/models) exhibit low rates of total destruction and low likelihood of fatal or serious passenger injuries in the event of an accident. They are also interested in any general variables/conditions that might be at play. Your analysis will be based off of aviation accident data accumulated from the years 1948-2023. 

Our client is only interested in airplane makes/models that are professional builds and could potentially still be active. Assume a max lifetime of 40 years for a make/model retirement and make sure to filter your data accordingly (i.e. from 1983 onwards). They would also like separate recommendations for small aircraft vs. larger passenger models. **In addition, make sure that claims that you make are statistically robust and that you have enough samples when making comparisons between groups.**


In this summative assessment you will demonstrate your ability to:
- **Use Pandas to load, inspect, and clean the dataset appropriately.**
- **Transform relevant columns to create measures that address the problem at hand.**
- conduct EDA: visualization and statistical measures to systematically understand the structure of the data
- recommend a set of airplanes and makes conforming to the client's request and identify at least *two* factors contributing to airplane safety. You must provide supporting evidence (visuals, summary statistics, tables) for each claim you make.

### Make relevant library imports

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Data Loading and Inspection

### Load in data from the relevant directory and inspect the dataframe.
- inspect NaNs, datatypes, and summary statistics

In [85]:
df = pd.read_csv('data/AviationData.csv', encoding='ISO-8859-1')

C:\Users\phill\AppData\Local\Temp\ipykernel_13328\2796400268.py:1: DtypeWarning: Columns (6,7,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/AviationData.csv', encoding='ISO-8859-1')


In [86]:
df.head()

Event.Id Investigation.Type Accident.Number  Event.Date  \
0  20001218X45444           Accident      SEA87LA080  1948-10-24   
1  20001218X45447           Accident      LAX94LA336  1962-07-19   
2  20061025X01555           Accident      NYC07LA005  1974-08-30   
3  20001218X45448           Accident      LAX96LA321  1977-06-19   
4  20041105X01764           Accident      CHI79FA064  1979-08-02   

          Location        Country   Latitude  Longitude Airport.Code  \
0  MOOSE CREEK, ID  United States        NaN        NaN          NaN   
1   BRIDGEPORT, CA  United States        NaN        NaN          NaN   
2    Saltville, VA  United States  36.922223 -81.878056          NaN   
3       EUREKA, CA  United States        NaN        NaN          NaN   
4       Canton, OH  United States        NaN        NaN          NaN   

  Airport.Name  ... Purpose.of.flight Air.carrier Total.Fatal.Injuries  \
0          NaN  ...          Personal         NaN                  2.0   
1          NaN  ...          Personal         NaN                  4.0   
2          NaN  ...          Personal         NaN                  3.0   
3          NaN  ...          Personal         NaN                  2.0   
4          NaN  ...          Personal         NaN                  1.0   

  Total.Serious.Injuries Total.Minor.Injuries Total.Uninjured  \
0                    0.0                  0.0             0.0   
1                    0.0                  0.0             0.0   
2                    NaN                  NaN             NaN   
3                    0.0                  0.0             0.0   
4                    2.0                  NaN             0.0   

  Weather.Condition  Broad.phase.of.flight   Report.Status Publication.Date  
0               UNK                 Cruise  Probable Cause              NaN  
1               UNK                Unknown  Probable Cause       19-09-1996  
2               IMC                 Cruise  Probable Cause       26-02-2007  
3               IMC                 Cruise  Probable Cause       12-09-2000  
4               VMC               Approach  Probable Cause       16-04-1980  

[5 rows x 31 columns]

In [87]:
df.dtypes

Event.Id                   object
Investigation.Type         object
Accident.Number            object
Event.Date                 object
Location                   object
Country                    object
Latitude                   object
Longitude                  object
Airport.Code               object
Airport.Name               object
Injury.Severity            object
Aircraft.damage            object
Aircraft.Category          object
Registration.Number        object
Make                       object
Model                      object
Amateur.Built              object
Number.of.Engines         float64
Engine.Type                object
FAR.Description            object
Schedule                   object
Purpose.of.flight          object
Air.carrier                object
Total.Fatal.Injuries      float64
Total.Serious.Injuries    float64
Total.Minor.Injuries      float64
Total.Uninjured           float64
Weather.Condition          object
Broad.phase.of.flight      object
Report.Status 

In [88]:
df.isna().sum()

Event.Id                      0
Investigation.Type            0
Accident.Number               0
Event.Date                    0
Location                     52
Country                     226
Latitude                  54507
Longitude                 54516
Airport.Code              38757
Airport.Name              36185
Injury.Severity            1000
Aircraft.damage            3194
Aircraft.Category         56602
Registration.Number        1382
Make                         63
Model                        92
Amateur.Built               102
Number.of.Engines          6084
Engine.Type                7096
FAR.Description           56866
Schedule                  76307
Purpose.of.flight          6192
Air.carrier               72241
Total.Fatal.Injuries      11401
Total.Serious.Injuries    12510
Total.Minor.Injuries      11933
Total.Uninjured            5912
Weather.Condition          4492
Broad.phase.of.flight     27165
Report.Status              6384
Publication.Date          13771
dtype: i

In [89]:
df.describe()

Number.of.Engines  Total.Fatal.Injuries  Total.Serious.Injuries  \
count       82805.000000          77488.000000            76379.000000   
mean            1.146585              0.647855                0.279881   
std             0.446510              5.485960                1.544084   
min             0.000000              0.000000                0.000000   
25%             1.000000              0.000000                0.000000   
50%             1.000000              0.000000                0.000000   
75%             1.000000              0.000000                0.000000   
max             8.000000            349.000000              161.000000   

       Total.Minor.Injuries  Total.Uninjured  
count          76956.000000     82977.000000  
mean               0.357061         5.325440  
std                2.235625        27.913634  
min                0.000000         0.000000  
25%                0.000000         0.000000  
50%                0.000000         1.000000  
75%                0.000000         2.000000  
max              380.000000       699.000000

## Data Cleaning

### Filtering aircrafts and events

We want to filter the dataset to include aircraft that the client is interested in an analysis of:
- inspect relevant columns
- figure out any reasonable imputations
- filter the dataset

In [90]:
df.columns

Index(['Event.Id', 'Investigation.Type', 'Accident.Number', 'Event.Date',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Injury.Severity', 'Aircraft.damage',
       'Aircraft.Category', 'Registration.Number', 'Make', 'Model',
       'Amateur.Built', 'Number.of.Engines', 'Engine.Type', 'FAR.Description',
       'Schedule', 'Purpose.of.flight', 'Air.carrier', 'Total.Fatal.Injuries',
       'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured',
       'Weather.Condition', 'Broad.phase.of.flight', 'Report.Status',
       'Publication.Date'],
      dtype='object')

In [91]:
#Dropping columns that are not needed for analysis
df = df.drop(['Event.Id', 'Investigation.Type', 'Accident.Number',
       'Location', 'Country', 'Latitude', 'Longitude', 'Airport.Code',
       'Airport.Name', 'Registration.Number', 'FAR.Description',
       'Schedule', 'Air.carrier', 'Report.Status', 'Publication.Date'], axis = 1)

In [92]:
df.head()

Event.Date Injury.Severity Aircraft.damage Aircraft.Category      Make  \
0  1948-10-24        Fatal(2)       Destroyed               NaN   Stinson   
1  1962-07-19        Fatal(4)       Destroyed               NaN     Piper   
2  1974-08-30        Fatal(3)       Destroyed               NaN    Cessna   
3  1977-06-19        Fatal(2)       Destroyed               NaN  Rockwell   
4  1979-08-02        Fatal(1)       Destroyed               NaN    Cessna   

      Model Amateur.Built  Number.of.Engines    Engine.Type Purpose.of.flight  \
0     108-3            No                1.0  Reciprocating          Personal   
1  PA24-180            No                1.0  Reciprocating          Personal   
2      172M            No                1.0  Reciprocating          Personal   
3       112            No                1.0  Reciprocating          Personal   
4       501            No                NaN            NaN          Personal   

   Total.Fatal.Injuries  Total.Serious.Injuries  Total.Minor.Injuries  \
0                   2.0                     0.0                   0.0   
1                   4.0                     0.0                   0.0   
2                   3.0                     NaN                   NaN   
3                   2.0                     0.0                   0.0   
4                   1.0                     2.0                   NaN   

   Total.Uninjured Weather.Condition Broad.phase.of.flight  
0              0.0               UNK                Cruise  
1              0.0               UNK               Unknown  
2              NaN               IMC                Cruise  
3              0.0               IMC                Cruise  
4              0.0               VMC              Approach

In [93]:
#Inspecting possibly relevent columns

In [94]:
import datetime

type(df['Event.Date'][0]) # need to convert to datetime

df['Event.Date'] = pd.to_datetime(df['Event.Date'])

In [95]:
#check column again
df.dtypes

Event.Date                datetime64[ns]
Injury.Severity                   object
Aircraft.damage                   object
Aircraft.Category                 object
Make                              object
Model                             object
Amateur.Built                     object
Number.of.Engines                float64
Engine.Type                       object
Purpose.of.flight                 object
Total.Fatal.Injuries             float64
Total.Serious.Injuries           float64
Total.Minor.Injuries             float64
Total.Uninjured                  float64
Weather.Condition                 object
Broad.phase.of.flight             object
dtype: object

In [96]:
'''Making the assumption that with only accident date and no info on production dates, we will filter out all accident dates older than 1983.
This will not omit all aircraft over 40 years old, but it is a reasonable basis given the data available.'''

df = df[df['Event.Date']>pd.to_datetime('1983-01-01')]

In [97]:
df.head()

Event.Date Injury.Severity Aircraft.damage Aircraft.Category  \
3606 1983-01-02       Non-Fatal             NaN               NaN   
3607 1983-01-02       Non-Fatal       Destroyed               NaN   
3608 1983-01-02        Fatal(2)       Destroyed               NaN   
3609 1983-01-02        Fatal(3)       Destroyed               NaN   
3610 1983-01-02       Non-Fatal     Substantial               NaN   

                Make       Model Amateur.Built  Number.of.Engines  \
3606   Balloon Works  FIREFLY 7B            No                0.0   
3607          Cessna        340A            No                2.0   
3608  North American        T-6G            No                1.0   
3609           Piper   PA-24-250            No                1.0   
3610           Piper  PA-32-301R            No                1.0   

        Engine.Type Purpose.of.flight  Total.Fatal.Injuries  \
3606        Unknown          Personal                   0.0   
3607  Reciprocating          Personal                   0.0   
3608  Reciprocating          Personal                   2.0   
3609  Reciprocating          Personal                   3.0   
3610  Reciprocating          Personal                   0.0   

      Total.Serious.Injuries  Total.Minor.Injuries  Total.Uninjured  \
3606                     1.0                   0.0              1.0   
3607                     0.0                   4.0              0.0   
3608                     0.0                   0.0              0.0   
3609                     0.0                   0.0              0.0   
3610                     0.0                   0.0              2.0   

     Weather.Condition Broad.phase.of.flight  
3606               VMC               Landing  
3607               VMC               Takeoff  
3608               VMC           Maneuvering  
3609               IMC                Cruise  
3610               VMC           Maneuvering

In [98]:
df['Aircraft.Category'].unique()

array([nan, 'Glider', 'Airplane', 'Helicopter', 'Blimp', 'Gyrocraft',
       'Balloon', 'Ultralight', 'Powered-Lift', 'Weight-Shift',
       'Powered Parachute', 'Unknown', 'Rocket', 'WSFT', 'UNK', 'ULTR'],
      dtype=object)

In [99]:
#Only looking at airplanes, keeping unknown/NaN for now to see if we can imply what the aircraft is via other columns
df = df[(df['Aircraft.Category'] == 'Airplane') | (df['Aircraft.Category'].isna()) | (df['Aircraft.Category'] == 'UNK') 
| (df['Aircraft.Category'] == 'Unknown') ]

In [100]:
df.head()
''' A quick google shows the Piper PA24-180 is an airplane. Need to figure out how to handle rows where Aircraft.Category is NaN, unknown, 
but first we will handle other columns. Let's at least make these values uniform for now'''

df['Aircraft.Category'] = df['Aircraft.Category'].replace([np.nan, 'UNK'], 'Unknown')
df['Aircraft.Category'].unique()

array(['Unknown', 'Airplane'], dtype=object)

In [101]:
df['Injury.Severity'].unique()

array(['Non-Fatal', 'Fatal(2)', 'Fatal(3)', 'Fatal(1)', 'Fatal(5)',
       'Fatal(7)', 'Incident', 'Fatal(6)', 'Fatal(4)', 'Fatal(23)',
       'Fatal(8)', 'Fatal(10)', 'Fatal(11)', 'Fatal(9)', 'Fatal(17)',
       'Fatal(13)', 'Fatal(29)', 'Fatal(70)', 'Unavailable', 'Fatal(135)',
       'Fatal(31)', 'Fatal(14)', 'Fatal(256)', 'Fatal(25)', 'Fatal(82)',
       'Fatal(156)', 'Fatal(28)', 'Fatal(18)', 'Fatal(43)', 'Fatal(15)',
       'Fatal(12)', 'Fatal(270)', 'Fatal(144)', 'Fatal(174)',
       'Fatal(111)', 'Fatal(131)', 'Fatal(20)', 'Fatal(73)', 'Fatal(27)',
       'Fatal(34)', 'Fatal(87)', 'Fatal(30)', 'Fatal(16)', 'Fatal(47)',
       'Fatal(56)', 'Fatal(37)', 'Fatal(132)', 'Fatal(68)', 'Fatal(54)',
       'Fatal(52)', 'Fatal(65)', 'Fatal(72)', 'Fatal(160)', 'Fatal(189)',
       'Fatal(123)', 'Fatal(33)', 'Fatal(110)', 'Fatal(230)', 'Fatal(97)',
       'Fatal(349)', 'Fatal(125)', 'Fatal(35)', 'Fatal(228)', 'Fatal(75)',
       'Fatal(104)', 'Fatal(229)', 'Fatal(80)', 'Fatal(217)',
      

In [102]:
# what are the values in Total.Fatal.Injuries, Total.Serious.Injuries, Total.Minor.Injuries, Total.Uninjured if Injury.Severity is 'nan'?
print('Total.Fatal.Injuries', df[df['Injury.Severity'].isna()]['Total.Fatal.Injuries'].unique())
print('Total.Serious.Injuries', df[df['Injury.Severity'].isna()]['Total.Serious.Injuries'].unique())
print('Total.Minor.Injuries', df[df['Injury.Severity'].isna()]['Total.Minor.Injuries'].unique())
print('Total.Uninjured', df[df['Injury.Severity'].isna()]['Total.Uninjured'].unique())

Total.Fatal.Injuries [0.]
Total.Serious.Injuries [0.]
Total.Minor.Injuries [0.]
Total.Uninjured [0.]


In [103]:
# 'nan' in Injury.Severity looks to imply no fatalies/injuries. We can imply 'None'
df['Injury.Severity'] = df['Injury.Severity'].fillna('None')
df['Injury.Severity'].value_counts()

Injury.Severity
Non-Fatal     61378
Fatal(1)       5846
Fatal          4370
Fatal(2)       3501
Incident       2094
              ...  
Fatal(80)         1
Fatal(217)        1
Fatal(169)        1
Fatal(88)         1
Fatal(123)        1
Name: count, Length: 108, dtype: int64

In [104]:
#Thinking about it, the injury severity column isn't necessary. All relevent injury info will be contained in the other columns
df = df.drop('Injury.Severity', axis = 1)

In [105]:
df.head()

Event.Date Aircraft.damage Aircraft.Category            Make       Model  \
3606 1983-01-02             NaN           Unknown   Balloon Works  FIREFLY 7B   
3607 1983-01-02       Destroyed           Unknown          Cessna        340A   
3608 1983-01-02       Destroyed           Unknown  North American        T-6G   
3609 1983-01-02       Destroyed           Unknown           Piper   PA-24-250   
3610 1983-01-02     Substantial           Unknown           Piper  PA-32-301R   

     Amateur.Built  Number.of.Engines    Engine.Type Purpose.of.flight  \
3606            No                0.0        Unknown          Personal   
3607            No                2.0  Reciprocating          Personal   
3608            No                1.0  Reciprocating          Personal   
3609            No                1.0  Reciprocating          Personal   
3610            No                1.0  Reciprocating          Personal   

      Total.Fatal.Injuries  Total.Serious.Injuries  Total.Minor.Injuries  \
3606                   0.0                     1.0                   0.0   
3607                   0.0                     0.0                   4.0   
3608                   2.0                     0.0                   0.0   
3609                   3.0                     0.0                   0.0   
3610                   0.0                     0.0                   0.0   

      Total.Uninjured Weather.Condition Broad.phase.of.flight  
3606              1.0               VMC               Landing  
3607              0.0               VMC               Takeoff  
3608              0.0               VMC           Maneuvering  
3609              0.0               IMC                Cruise  
3610              2.0               VMC           Maneuvering

In [106]:
df['Make'] = df['Make'].str.strip().str.title()

'''Making some assumptions here. Without much industry knowledge, we cannot assume overlap of certain makes, i.e. should 'Grumman American' 
be considered the same make as 'Grumman-Schweizer', etc.? We will keep them seperate for the purpose of this analysis, outside of a few
exceptions. We will also shortern some names.
'''

value_map = {
    'Grumman Acft Eng Cor-Schweizer': 'Grumman-Schweizer',
    'Smith, Ted Aerostar': 'Aerostar',
    'Ercoupe (Eng & Research Corp.)': 'Ercoupe',
    'Boeing Stearman': 'Boeing',
    'Cirrus Design Corp.': 'Cirrus',
    'Cirrus Design Corp': 'Cirrus',
    'Rockwell International': 'Rockwell',
    'Raytheon Aircraft Company': 'Raytheon',
    'American Champion Aircraft': 'Champion',
    'Air Tractor Inc': 'Air Tractor',
    'Diamond Aircraft Ind Inc': 'Diamond',
    'Airbus Industrie': 'Airbus',
    'Aviat Aircraft Inc': 'Aviat'
}

df['Make'] = df['Make'].replace(value_map)

In [107]:
df['Amateur.Built'].unique()

array(['No', 'Yes', nan], dtype=object)

In [108]:
#We want pro built aircraft. We can take out 'Yes's but is it reasonable to imply a value from NaN?
df[df['Amateur.Built'].isna()].head()

Event.Date Aircraft.damage Aircraft.Category        Make  \
47562 1999-12-06       Destroyed          Airplane       Piper   
48241 2000-04-22             NaN          Airplane      Boeing   
49732 2000-11-22       Destroyed           Unknown  2000 Mccoy   
50016 2001-01-30             NaN          Airplane      Boeing   
50019 2001-01-31     Substantial          Airplane      Cessna   

                   Model Amateur.Built  Number.of.Engines    Engine.Type  \
47562           PA31-350           NaN                NaN        Unknown   
48241            747-300           NaN                NaN            NaN   
49732            Genesis           NaN                1.0  Reciprocating   
50016          B-777-300           NaN                NaN            NaN   
50019  CitationJET (525)           NaN                2.0      Turbo Fan   

      Purpose.of.flight  Total.Fatal.Injuries  Total.Serious.Injuries  \
47562          Business                  10.0                     NaN   
48241               NaN                   NaN                     NaN   
49732       Flight Test                   1.0                     NaN   
50016               NaN                   NaN                     NaN   
50019               NaN                   NaN                     NaN   

       Total.Minor.Injuries  Total.Uninjured Weather.Condition  \
47562                   NaN              NaN               VMC   
48241                   NaN              NaN               NaN   
49732                   NaN              NaN               VMC   
50016                   NaN              NaN               NaN   
50019                   NaN              7.0               NaN   

      Broad.phase.of.flight  
47562                   NaN  
48241                   NaN  
49732                Cruise  
50016                   NaN  
50019                   NaN

In [109]:
df[df['Amateur.Built'] == 'Yes'].head()

Event.Date Aircraft.damage Aircraft.Category               Make    Model  \
3625 1983-01-04       Destroyed           Unknown   Javelin Aircraft  WICHAWK   
3635 1983-01-05     Substantial           Unknown             Cessna    T210L   
3664 1983-01-09       Destroyed           Unknown              Rutan  VARIEZE   
3665 1983-01-09       Destroyed           Unknown              Beech      D55   
3669 1983-01-09     Substantial           Unknown  Mcdonnell Douglas      F4C   

     Amateur.Built  Number.of.Engines    Engine.Type Purpose.of.flight  \
3625           Yes                1.0  Reciprocating          Personal   
3635           Yes                1.0  Reciprocating          Personal   
3664           Yes                1.0  Reciprocating          Personal   
3665           Yes                2.0  Reciprocating          Personal   
3669           Yes                2.0      Turbo Jet   Public Aircraft   

      Total.Fatal.Injuries  Total.Serious.Injuries  Total.Minor.Injuries  \
3625                   2.0                     0.0                   0.0   
3635                   0.0                     0.0                   0.0   
3664                   0.0                     1.0                   0.0   
3665                   7.0                     0.0                   0.0   
3669                   7.0                     0.0                   0.0   

      Total.Uninjured Weather.Condition Broad.phase.of.flight  
3625              0.0               VMC           Maneuvering  
3635              2.0               VMC                Cruise  
3664              0.0               VMC              Approach  
3665              2.0               VMC                Cruise  
3669              2.0               VMC           Maneuvering

In [110]:
df[df['Amateur.Built'] == 'No'].head()

Event.Date Aircraft.damage Aircraft.Category            Make       Model  \
3606 1983-01-02             NaN           Unknown   Balloon Works  FIREFLY 7B   
3607 1983-01-02       Destroyed           Unknown          Cessna        340A   
3608 1983-01-02       Destroyed           Unknown  North American        T-6G   
3609 1983-01-02       Destroyed           Unknown           Piper   PA-24-250   
3610 1983-01-02     Substantial           Unknown           Piper  PA-32-301R   

     Amateur.Built  Number.of.Engines    Engine.Type Purpose.of.flight  \
3606            No                0.0        Unknown          Personal   
3607            No                2.0  Reciprocating          Personal   
3608            No                1.0  Reciprocating          Personal   
3609            No                1.0  Reciprocating          Personal   
3610            No                1.0  Reciprocating          Personal   

      Total.Fatal.Injuries  Total.Serious.Injuries  Total.Minor.Injuries  \
3606                   0.0                     1.0                   0.0   
3607                   0.0                     0.0                   4.0   
3608                   2.0                     0.0                   0.0   
3609                   3.0                     0.0                   0.0   
3610                   0.0                     0.0                   0.0   

      Total.Uninjured Weather.Condition Broad.phase.of.flight  
3606              1.0               VMC               Landing  
3607              0.0               VMC               Takeoff  
3608              0.0               VMC           Maneuvering  
3609              0.0               IMC                Cruise  
3610              2.0               VMC           Maneuvering

In [111]:
print(df[df['Amateur.Built'] == 'No']['Make'].value_counts())
print(df[df['Amateur.Built'].isna()]['Make'].value_counts())
print(df[df['Amateur.Built'] == 'Yes']['Make'].value_counts())

Make
Cessna                            25736
Piper                             14098
Beech                              5094
Boeing                             2696
Bell                               1795
                                  ...  
Emory B. Wood                         1
Aerofab                               1
Eagle (Ultralight)                    1
Extra Flugzeugproduktions-Gmbh        1
Royse Ralph L                         1
Name: count, Length: 1544, dtype: int64
Make
Boeing                     33
Cessna                     18
Piper                       8
Mcdonnell Douglas           6
Beech                       4
Airbus                      4
British Aerospace           3
Raytheon Corporate Jets     3
Lockheed                    2
Hebb                        1
Rockwell                    1
Eurocopter                  1
Bell                        1
Douglas                     1
Lake                        1
Robinson                    1
Westland Helicopters        1


In [112]:
ct = pd.crosstab(df['Make'], df['Amateur.Built'], dropna=False)
ct.head()

Amateur.Built             No  Yes  NaN
Make                                  
107.5 Flying Corporation   0    1    0
177Mf Llc                  1    0    0
1977 Colfer-Chan           0    1    0
1St Ftr Gp                 0    1    0
2000 Mccoy                 0    0    1

In [114]:
ct.columns = ['No', 'Yes', 'N/A']

In [116]:
ct[(ct['Yes'] == 0) & (ct['N/A'] > 0)]

No  Yes  N/A
Make                                  
2000 Mccoy                 0    0    1
British Aerospace         87    0    3
Casa                      21    0    1
Dassault                  29    0    1
Eurocopter                71    0    1
Hebb                       0    0    1
Lake                     138    0    1
Raytheon Corporate Jets   13    0    3
Saab-Scania Ab (Saab)     43    0    1
Schleicher                72    0    1
Socata                   125    0    1
Tecnam                    13    0    1
Westland Helicopters       3    0    1

In [117]:
'''If all amateur.built values are 'No' or 'NaN' for a specific make (without any 'Yes' values that is), we are implying "No' for 
NaN values of this make. This does make an assumption that all these aircraft are pro built.'''

#Find makes that are only amateur.built = 'No' or 'NaN'
makes_always_no = (
    df[df['Amateur.Built'].notna()]
    .groupby('Make')['Amateur.Built']
    .apply(lambda x: (x == 'No').all())
)

makes_with_only_no = makes_always_no[makes_always_no].index

#Change these values to 'No'
df.loc[(df['Make'].isin(makes_with_only_no)) & (df['Amateur.Built'].isna()), 'Amateur.Built'] = 'No'

In [126]:
#Get rid of amateur built aircraft
df = df[df['Amateur.Built'] == 'No']
df['Amateur.Built'].unique()

array(['No'], dtype=object)

In [127]:
#Looking at Makes
df['Make'].value_counts()

#Let's just take out nan. These rows do not help recommend an aircraft. Same with model.

df = df[df['Make'].notna()]
df = df[df['Model'].notna()]

In [131]:
'''Running a similar filter on aircraft category as we did with Amateur.Built. If the maker appears to only make airplanes based 
on this data, it implies 'airplane' for any unknown values under that make.''' 

aircraft_always_airplane = (
    df[df['Aircraft.Category'] != 'Unknown']
    .groupby('Make')['Aircraft.Category']
    .apply(lambda x: (x == 'Airplane').all())
)

aircraft_only = aircraft_always_airplane[aircraft_always_airplane].index

df.loc[(df['Make'].isin(aircraft_only)) & (df['Aircraft.Category'] == 'Unknown'), 'Aircraft.Category'] = 'Airplane'
df = df[df['Aircraft.Category'] == 'Airplane']

In [132]:
df.head()

Event.Date Aircraft.damage Aircraft.Category            Make       Model  \
3607 1983-01-02       Destroyed          Airplane          Cessna        340A   
3608 1983-01-02       Destroyed          Airplane  North American        T-6G   
3609 1983-01-02       Destroyed          Airplane           Piper   PA-24-250   
3610 1983-01-02     Substantial          Airplane           Piper  PA-32-301R   
3611 1983-01-02     Substantial          Airplane           Beech       V-35B   

     Amateur.Built  Number.of.Engines    Engine.Type Purpose.of.flight  \
3607            No                2.0  Reciprocating          Personal   
3608            No                1.0  Reciprocating          Personal   
3609            No                1.0  Reciprocating          Personal   
3610            No                1.0  Reciprocating          Personal   
3611            No                1.0  Reciprocating          Personal   

      Total.Fatal.Injuries  Total.Serious.Injuries  Total.Minor.Injuries  \
3607                   0.0                     0.0                   4.0   
3608                   2.0                     0.0                   0.0   
3609                   3.0                     0.0                   0.0   
3610                   0.0                     0.0                   0.0   
3611                   0.0                     0.0                   0.0   

      Total.Uninjured Weather.Condition Broad.phase.of.flight  
3607              0.0               VMC               Takeoff  
3608              0.0               VMC           Maneuvering  
3609              0.0               IMC                Cruise  
3610              2.0               VMC           Maneuvering  
3611              3.0               VMC               Takeoff

In [133]:
df['Number.of.Engines'].unique()

array([ 2.,  1.,  3.,  4., nan,  0.,  8.,  6.])

In [134]:
df['Engine.Type'].unique()

array(['Reciprocating', 'Turbo Prop', 'Turbo Fan', 'Turbo Shaft',
       'Turbo Jet', 'Unknown', nan, 'Electric', 'Geared Turbofan', 'UNK'],
      dtype=object)

In [136]:
'''If any injury metrics are NaN, it seems reasonable to imply that the number is 0. It is much more likely these values were implied zero
than a fatality, etc. that was not recorded
'''

df[['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']] = df[['Total.Fatal.Injuries', 'Total.Serious.Injuries', 'Total.Minor.Injuries', 'Total.Uninjured']].fillna(0)


In [137]:
df.head()

Event.Date Aircraft.damage Aircraft.Category            Make       Model  \
3607 1983-01-02       Destroyed          Airplane          Cessna        340A   
3608 1983-01-02       Destroyed          Airplane  North American        T-6G   
3609 1983-01-02       Destroyed          Airplane           Piper   PA-24-250   
3610 1983-01-02     Substantial          Airplane           Piper  PA-32-301R   
3611 1983-01-02     Substantial          Airplane           Beech       V-35B   

     Amateur.Built  Number.of.Engines    Engine.Type Purpose.of.flight  \
3607            No                2.0  Reciprocating          Personal   
3608            No                1.0  Reciprocating          Personal   
3609            No                1.0  Reciprocating          Personal   
3610            No                1.0  Reciprocating          Personal   
3611            No                1.0  Reciprocating          Personal   

      Total.Fatal.Injuries  Total.Serious.Injuries  Total.Minor.Injuries  \
3607                   0.0                     0.0                   4.0   
3608                   2.0                     0.0                   0.0   
3609                   3.0                     0.0                   0.0   
3610                   0.0                     0.0                   0.0   
3611                   0.0                     0.0                   0.0   

      Total.Uninjured Weather.Condition Broad.phase.of.flight  
3607              0.0               VMC               Takeoff  
3608              0.0               VMC           Maneuvering  
3609              0.0               IMC                Cruise  
3610              2.0               VMC           Maneuvering  
3611              3.0               VMC               Takeoff

In [138]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 69467 entries, 3607 to 88888
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Event.Date              69467 non-null  datetime64[ns]
 1   Aircraft.damage         66793 non-null  object        
 2   Aircraft.Category       69467 non-null  object        
 3   Make                    69467 non-null  object        
 4   Model                   69467 non-null  object        
 5   Amateur.Built           69467 non-null  object        
 6   Number.of.Engines       65102 non-null  float64       
 7   Engine.Type             64386 non-null  object        
 8   Purpose.of.flight       64257 non-null  object        
 9   Total.Fatal.Injuries    69467 non-null  float64       
 10  Total.Serious.Injuries  69467 non-null  float64       
 11  Total.Minor.Injuries    69467 non-null  float64       
 12  Total.Uninjured         69467 non-null  float64 

### Cleaning and constructing Key Measurables

Injuries and robustness to destruction are a key interest point for the client. Clean and impute relevant columns and then create derived fields that best quantifies what the client wishes to track. **Use commenting or markdown to explain any cleaning assumptions as well as any derived columns you create.**

**Construct metric for fatal/serious injuries**

*Hint:* Estimate the total number of passengers on each flight. The likelihood of serious / fatal injury can be estimated as a fraction from this.

In [139]:
#We can imply that total passangers is simply the sum of Total.Fatal.Injuries, Total.Serious.Injuries, Total.Minor.Injuries and Total.Uninjured

df['Total Passangers'] = df.loc[:,'Total.Fatal.Injuries':'Total.Uninjured'].sum(axis=1)

df['Fatal/Serious Injury Chance'] = (df.loc[:,'Total.Fatal.Injuries':'Total.Serious.Injuries'].sum(axis=1))/(df['Total Passangers'])

'''If df['Fatal/Serious Injury Chance'] is NaN, i.e. div zero error, then total passangers must be zero. These rows should be omitted as the flight
was either not recorded properly or unmanned and therefore not helpful to the analysis'''

df = df[df['Fatal/Serious Injury Chance'].notna()]

**Aircraft.Damage**
- identify and execute any cleaning tasks
- construct a derived column tracking whether an aircraft was destroyed or not.

In [140]:
df['Aircraft.damage'].value_counts()

Aircraft.damage
Substantial    50011
Destroyed      14054
Minor           2068
Unknown           74
Name: count, dtype: int64

In [141]:
#Any NaN or 'Unknown' values will not be useful to us.

df = df[(df['Aircraft.damage'] != 'Unknown') & (df['Aircraft.damage'].notna())]

df['Aircraft.damage'].value_counts()

Aircraft.damage
Substantial    50011
Destroyed      14054
Minor           2068
Name: count, dtype: int64

In [142]:
#If df['Aircraft.damage'] is 'Destroyed', this new column will be true, otherwise false.

df['Destroyed?'] = df['Aircraft.damage'] == 'Destroyed'

In [144]:
df.head()

Event.Date Aircraft.damage Aircraft.Category            Make       Model  \
3607 1983-01-02       Destroyed          Airplane          Cessna        340A   
3608 1983-01-02       Destroyed          Airplane  North American        T-6G   
3609 1983-01-02       Destroyed          Airplane           Piper   PA-24-250   
3610 1983-01-02     Substantial          Airplane           Piper  PA-32-301R   
3611 1983-01-02     Substantial          Airplane           Beech       V-35B   

     Amateur.Built  Number.of.Engines    Engine.Type Purpose.of.flight  \
3607            No                2.0  Reciprocating          Personal   
3608            No                1.0  Reciprocating          Personal   
3609            No                1.0  Reciprocating          Personal   
3610            No                1.0  Reciprocating          Personal   
3611            No                1.0  Reciprocating          Personal   

      Total.Fatal.Injuries  Total.Serious.Injuries  Total.Minor.Injuries  \
3607                   0.0                     0.0                   4.0   
3608                   2.0                     0.0                   0.0   
3609                   3.0                     0.0                   0.0   
3610                   0.0                     0.0                   0.0   
3611                   0.0                     0.0                   0.0   

      Total.Uninjured Weather.Condition Broad.phase.of.flight  \
3607              0.0               VMC               Takeoff   
3608              0.0               VMC           Maneuvering   
3609              0.0               IMC                Cruise   
3610              2.0               VMC           Maneuvering   
3611              3.0               VMC               Takeoff   

      Total Passangers  Fatal/Serious Injury Chance  Destroyed?  
3607               4.0                          0.0        True  
3608               2.0                          1.0        True  
3609               3.0                          1.0        True  
3610               2.0                          0.0       False  
3611               3.0                          0.0       False

### Investigate the *Make* column
- Identify cleaning tasks here
- List cleaning tasks clearly in markdown
- Execute the cleaning tasks
- For your analysis, keep Makes with a reasonable number (you can put the threshold at 50 though lower could work as well)

In [145]:
# We did most cleaning above already, but let's set a threshold.
value_counts = df['Make'].value_counts()

df = df[df['Make'].isin(value_counts[value_counts >= 50].index)]

df['Make'].unique()

array(['Cessna', 'North American', 'Piper', 'Beech', 'Swearingen',
       'Canadair', 'Douglas', 'Mooney', 'Bell', 'Hiller', 'Bellanca',
       'Grumman', 'Luscombe', 'Mcdonnell Douglas', 'Lockheed', 'Champion',
       'Grumman American', 'Maule', 'Boeing', 'Aero Commander',
       'Mitsubishi', 'Stinson', 'Schweizer', 'Aerostar', 'Gates Learjet',
       'Rockwell', 'Taylorcraft', 'Gulfstream', 'Embraer', 'Aeronca',
       'Navion', 'Aerospatiale', 'Helio', 'De Havilland', 'Ayres', 'Ryan',
       'American', 'Air Tractor', 'Let', 'Ercoupe', 'Great Lakes',
       'Weatherly', 'Globe', 'Waco', 'Lake', 'Grumman-Schweizer',
       'Learjet', 'Pitts', 'Fairchild', 'Wsk Pzl Mielec', 'Pilatus',
       'Airbus', 'British Aerospace', 'Cirrus', 'Socata', 'Aviat',
       'Raytheon', 'Dehavilland', 'Flight Design Gmbh', 'Diamond'],
      dtype=object)

### Inspect Model column
- Get rid of any NaNs.
- Inspect the column and counts for each model/make. Are model labels unique to each make?
- If not, create a derived column that is a unique identifier for a given plane type.

In [146]:
print(df['Make'].value_counts())
print(df['Model'].value_counts())

Make
Cessna                25380
Piper                 13934
Beech                  5000
Bell                   1741
Boeing                 1350
Mooney                 1256
Grumman                1055
Bellanca                973
Air Tractor             882
Aeronca                 604
Maule                   569
Champion                554
Schweizer               543
Cirrus                  431
Stinson                 421
Aero Commander          405
Rockwell                403
Mcdonnell Douglas       394
Luscombe                386
North American          366
Taylorcraft             364
De Havilland            363
Hiller                  279
Aerospatiale            264
Ercoupe                 232
Ayres                   225
Aviat                   216
Grumman American        211
Douglas                 211
Grumman-Schweizer       184
Swearingen              145
Embraer                 143
Airbus                  138
Lake                    138
Waco                    138
Fairchild      

In [147]:
make_model = df.groupby('Model')['Make'].nunique()
overlap = make_model[make_model > 1]
overlap
#Clearly there is some overlap between model and makes.

Model
100                  3
100-180              2
112                  3
112A                 2
112TCA               2
                    ..
TB20                 2
TB21                 2
THRUSH S2R           3
TIGER MOTH DH 82A    2
V35A                 2
Name: Make, Length: 270, dtype: int64

In [150]:
df['Type'] = df['Make'] + ' ' + df['Model']

df.head()

Event.Date Aircraft.damage Aircraft.Category            Make       Model  \
3607 1983-01-02       Destroyed          Airplane          Cessna        340A   
3608 1983-01-02       Destroyed          Airplane  North American        T-6G   
3609 1983-01-02       Destroyed          Airplane           Piper   PA-24-250   
3610 1983-01-02     Substantial          Airplane           Piper  PA-32-301R   
3611 1983-01-02     Substantial          Airplane           Beech       V-35B   

     Amateur.Built  Number.of.Engines    Engine.Type Purpose.of.flight  \
3607            No                2.0  Reciprocating          Personal   
3608            No                1.0  Reciprocating          Personal   
3609            No                1.0  Reciprocating          Personal   
3610            No                1.0  Reciprocating          Personal   
3611            No                1.0  Reciprocating          Personal   

      Total.Fatal.Injuries  Total.Serious.Injuries  Total.Minor.Injuries  \
3607                   0.0                     0.0                   4.0   
3608                   2.0                     0.0                   0.0   
3609                   3.0                     0.0                   0.0   
3610                   0.0                     0.0                   0.0   
3611                   0.0                     0.0                   0.0   

      Total.Uninjured Weather.Condition Broad.phase.of.flight  \
3607              0.0               VMC               Takeoff   
3608              0.0               VMC           Maneuvering   
3609              0.0               IMC                Cruise   
3610              2.0               VMC           Maneuvering   
3611              3.0               VMC               Takeoff   

      Total Passangers  Fatal/Serious Injury Chance  Destroyed?  \
3607               4.0                          0.0        True   
3608               2.0                          1.0        True   
3609               3.0                          1.0        True   
3610               2.0                          0.0       False   
3611               3.0                          0.0       False   

                     Type  
3607          Cessna 340A  
3608  North American T-6G  
3609      Piper PA-24-250  
3610     Piper PA-32-301R  
3611          Beech V-35B

### Cleaning other columns
- there are other columns containing data that might be related to the outcome of an accident. We list a few here:
- Engine.Type
- Weather.Condition
- Number.of.Engines
- Purpose.of.flight
- Broad.phase.of.flight

Inspect and identify potential cleaning tasks in each of the above columns. Execute those cleaning tasks. 

**Note**: You do not necessarily need to impute or drop NaNs here.

In [151]:
#Imply 'Unknown' for all NaN, 'UNK'

df['Engine.Type'] = df['Engine.Type'].replace([np.nan, 'UNK'], 'Unknown')

df['Weather.Condition'] = df['Weather.Condition'].replace([np.nan, 'UNK', 'Unk'], 'Unknown')

df['Number.of.Engines'] = df['Number.of.Engines'].replace([np.nan], 'Unknown')

#Condensing some of these purpose values that make reasonable sense. Assuming PUBS is public service and ASHO is air show
df['Purpose.of.flight'] = df['Purpose.of.flight'].replace([np.nan], 'Unknown')
df['Purpose.of.flight'] = df['Purpose.of.flight'].replace(['Public Aircraft - Federal',
                            'Public Aircraft - Local','Public Aircraft - State', 'Firefighting', 'PUBS'], 'Public Aircraft')
df['Purpose.of.flight'] = df['Purpose.of.flight'].replace(['Executive/corporate'],'Business')
df['Purpose.of.flight'] = df['Purpose.of.flight'].replace(['Air Race/show','ASHO'],'Air Race show')

df['Broad.phase.of.flight'] = df['Broad.phase.of.flight'].replace([np.nan, 'Other'], 'Unknown')

df['Broad.phase.of.flight'].unique()

array(['Takeoff', 'Maneuvering', 'Cruise', 'Landing', 'Climb',
       'Go-around', 'Approach', 'Taxi', 'Descent', 'Unknown', 'Standing'],
      dtype=object)

### Column Removal
- inspect the dataframe and drop any columns that have too many NaNs

In [152]:
#No NaN values remaining, dataframe looks ready for analysis
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 62119 entries, 3607 to 88886
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Event.Date                   62119 non-null  datetime64[ns]
 1   Aircraft.damage              62119 non-null  object        
 2   Aircraft.Category            62119 non-null  object        
 3   Make                         62119 non-null  object        
 4   Model                        62119 non-null  object        
 5   Amateur.Built                62119 non-null  object        
 6   Number.of.Engines            62119 non-null  object        
 7   Engine.Type                  62119 non-null  object        
 8   Purpose.of.flight            62119 non-null  object        
 9   Total.Fatal.Injuries         62119 non-null  float64       
 10  Total.Serious.Injuries       62119 non-null  float64       
 11  Total.Minor.Injuries         62119 non-null

### Save DataFrame to csv
- its generally useful to save data to file/server after its in a sufficiently cleaned or intermediate state
- the data can then be loaded directly in another notebook for further analysis
- this helps keep your notebooks and workflow readable, clean and modularized

In [153]:
df.to_csv('data/CleanedAviationData.csv', index=False)